In [31]:
import numpy as np
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Flatten, Dropout
from sklearn.model_selection import StratifiedKFold
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

# Rede Neural - Convolucional

In [2]:
# Importando base de dados
(X_treinamento, y_treinamento), (X_teste, y_teste) = mnist.load_data()

In [3]:
X_treinamento.shape, y_treinamento.shape, X_teste.shape, y_teste.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [9]:
# Realizando conversão, reshape e scaling dos valores
previsores_treinamento = X_treinamento.reshape(X_treinamento.shape[0], 28, 28, 1)
previsores_treinamento = previsores_treinamento.astype('float32')
previsores_treinamento /= 255

previsores_teste = X_teste.reshape(X_teste.shape[0], 28, 28, 1)
previsores_teste = previsores_teste.astype('float32')
previsores_teste /= 255

In [10]:
previsores_treinamento.shape, previsores_teste.shape

((60000, 28, 28, 1), (10000, 28, 28, 1))

In [11]:
# Realizando One Hot Encoding nas classes 
classe_treinamento = np_utils.to_categorical(y_treinamento, 10)
classe_teste = np_utils.to_categorical(y_teste, 10)

In [12]:
y_treinamento.shape, y_teste.shape

((60000,), (10000,))

In [13]:
classe_treinamento.shape, classe_teste.shape

((60000, 10), (10000, 10))

In [8]:
# Criando estrutura da rede neural
classificador = Sequential()
classificador.add(Conv2D(32, (3, 3), input_shape=(28, 28, 1), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2, 2)))
classificador.add(Conv2D(32, (3, 3), activation = 'relu'))
classificador.add(BatchNormalization())
classificador.add(MaxPooling2D(pool_size = (2, 2)))
classificador.add(Flatten())
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dropout(0.2))
classificador.add(Dense(units = 10, activation = 'softmax'))
# Compilando rede neural
classificador.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [9]:
# Resumo do modelo criado
classificador.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
batch_normalization_2 (Batch (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 32)        9248      
_________________________________________________________________
batch_normalization_3 (Batch (None, 11, 11, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 800)              

In [10]:
# Treinamento e teste do modelo
classificador.fit(
    previsores_treinamento, 
    classe_treinamento, 
    batch_size = 128, 
    epochs = 2, 
    validation_data = (previsores_teste, classe_teste)
)

Epoch 1/2
469/469 [==============================] - 44s 91ms/step - loss: 0.4297 - accuracy: 0.8663 - val_loss: 0.1897 - val_accuracy: 0.9415
Epoch 2/2
469/469 [==============================] - 39s 84ms/step - loss: 0.0646 - accuracy: 0.9808 - val_loss: 0.0392 - val_accuracy: 0.9867


In [11]:
# Accuracy usando keras
resultado = classificador.evaluate(previsores_teste, classe_teste)

313/313 [==============================] - 3s 8ms/step - loss: 0.0392 - accuracy: 0.9867


# Rede Neural - Convolucional
## Validação Cruzada com K-fold 

In [17]:
# Definindo semente
seed = 5
np.random.seed(seed)

In [18]:
# Importando base de dados
(X, y), (X_teste, y_teste) = mnist.load_data()

# Realizando conversão, reshape e scaling dos valores
previsores = X.reshape(X.shape[0], 28, 28, 1)
previsores = previsores.astype('float32')
previsores /= 255

# Realizando One Hot Encoding nas classes 
classe = np_utils.to_categorical(y, 10)

In [19]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)
resultados = []

#b = np.zeros(shape = (classe.shape[0], 1))

In [20]:
for indice_treinamento, indice_teste in kfold.split(previsores, np.zeros(shape = (classe.shape[0], 1))):
    #print('Índices treinamento: ', indice_treinamento, 'Índice teste', indice_teste)

    # Criando estrutura da rede neural
    classificador = Sequential()
    classificador.add(Conv2D(32, (3,3), input_shape=(28,28,1), activation = 'relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())
    classificador.add(Dense(units = 128, activation = 'relu'))
    classificador.add(Dense(units = 10, activation = 'softmax'))
    # Compilando rede neural
    classificador.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    # Treinamento e teste do modelo
    classificador.fit(previsores[indice_treinamento], classe[indice_treinamento], batch_size = 128, epochs = 5)

    # Accuracy usando keras
    precisao = classificador.evaluate(previsores[indice_teste], classe[indice_teste])
    resultados.append(precisao[1])

Epoch 1/5
375/375 [==============================] - 14s 36ms/step - loss: 0.4893 - accuracy: 0.8633
Epoch 2/5
375/375 [==============================] - 14s 39ms/step - loss: 0.0847 - accuracy: 0.9761
Epoch 3/5
375/375 [==============================] - 13s 35ms/step - loss: 0.0520 - accuracy: 0.9842
Epoch 4/5
375/375 [==============================] - 14s 38ms/step - loss: 0.0328 - accuracy: 0.9912
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 0.0630 - accuracy: 0.9807
Epoch 1/5
375/375 [==============================] - 14s 35ms/step - loss: 0.5244 - accuracy: 0.8506
Epoch 2/5
375/375 [==============================] - 13s 35ms/step - loss: 0.0913 - accuracy: 0.9739
Epoch 3/5
375/375 [==============================] - 15s 40ms/step - loss: 0.0524 - accuracy: 0.9851
Epoch 4/5
375/375 [==============================] - 13s 34ms/step - loss: 0.0389 - accuracy: 0.9884
Epoch 5/5
375/375 [==============================] - 1s 3ms/step - loss: 0.0641 - accuracy: 0

In [30]:
resultados

[0.9806666374206543,
 0.9806666374206543,
 0.9848333597183228,
 0.9831666946411133,
 0.9830833077430725]

In [22]:
# media = resultados.mean()
media = sum(resultados) / len(resultados)
media

0.9824833273887634

# Rede Neural - Convolucional
## Augumentation

In [33]:
# Criando estrutura da rede neural
classificador = Sequential()
classificador.add(Conv2D(32, (3,3), input_shape=(28, 28, 1), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2,2)))
classificador.add(Flatten())
classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 10, activation = 'softmax'))
# Compilando rede neural
classificador.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

gerador_treinamento = ImageDataGenerator(
    rotation_range = 7,
    horizontal_flip = True,
    shear_range = 0.2,
    height_shift_range = 0.07,
    zoom_range = 0.2
)

# Parâmetros do Augumentation
gerador_teste = ImageDataGenerator()
base_treinamento = gerador_treinamento.flow(previsores_treinamento, classe_treinamento, batch_size = 128)
base_teste = gerador_teste.flow(previsores_teste, classe_teste, batch_size = 128)

# Treinamento e teste do modelo
classificador.fit_generator(
    base_treinamento, 
    steps_per_epoch = 600000 // 128,
    epochs = 5, 
    validation_data = base_teste,
    validation_steps = 10000 // 128
)

Epoch 1/5
4687/4687 [==============================] - 34s 7ms/step - loss: 0.5866 - accuracy: 0.8129 - val_loss: 0.1743 - val_accuracy: 0.9486
